# Попытка улучшить показатели лучшей реализации с ошибкой в ~12.93% RMSE
Прошлый подход, где пытался сделать ансамбли - не состоялся. Он показал интересные результаты на простых линейных моделях, которые можно включать в ансамбль (стекингом, например) - и да, там результаты значительно лучше, чем у простой линейной модели. При попытке же использовать catboost  в бегинге или стекинге - ничего хорошего не было. Результаты улучшить не удалось. В общем-то потому, что catboost сам по себе уже ансамблевый подход реализует - это раз. В процессе интеграции в ансамбль - мы учим модель через тот же катбуст, но на какой-то усеченной пропорции данных и фич, как итог, получаем множество более слабых моделей, ансамбль усредняет все это дело и поэтому на выходе не можем получить ничего дельного.

Сейчас мы попробуем вернуться к improved_realization, где мы получили показатели
RMSE: 23271.3399
R² (коэффициент детерминации): 0.9224
Относительная ошибка (RMSE/mean): 12.93%
И используя optuna для подбора гиперпараметров попробуем улучшить результаты.

In [1]:
from IPython.lib.deepreload import reload
from xgboost import train
%load_ext autoreload
%autoreload 2

import joblib
import numpy as np
import optuna
import pandas as pd

from catboost import CatBoostRegressor
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, PowerTransformer, RobustScaler, QuantileTransformer, PolynomialFeatures, KBinsDiscretizer

# Вместо одного фиксированного разбиения на train/test используем стабильную стратегию кросс-валидации.
# Используем тут Cross-validation, потому что:
# 	•	нужно надёжно сравнить несколько разных моделей или гиперпараметров и понять, какая модель стабильнее и лучше в целом.
# 	•	хотим избежать случайных удач или провалов, связанных с конкретным разбиением на train/test.
# 	•	выбираем модель или гиперпараметры, которые потом будешь использовать для финального сабмишна на Kaggle.
# Делаем эту оценку, чтобы в дальнейших блокнотах-улучшениях сравнивать более корректно.
from sklearn.model_selection import KFold, cross_val_score, train_test_split

from utils.data_manager import DataManager
from utils.model_manager import ModelManager

## 1. Загрузка данных

In [2]:
data_path = 'data/home-data-for-ml-course'
train_data = pd.read_csv(data_path + '/train.csv')
test_data = pd.read_csv(data_path + '/test.csv')

## 2. Предобработка данных 

In [3]:
dm = DataManager()

### 2.1. Удаление ненужных столбцов

In [4]:
intuitively_bad_features = [
    'LotShape',  # Общая форма участка
    'LandContour',  # Рельеф участка
    'LotConfig',  # Конфигурация участка
    'LandSlope',  # Уклон участка
    'MiscFeature',
    'MiscVal',
]
bad_columns = dm.get_all_nan_cols(train_data)
bad_columns.append('Id')
bad_columns.extend(intuitively_bad_features)
bad_columns

['Id',
 'LotShape',
 'LandContour',
 'LotConfig',
 'LandSlope',
 'MiscFeature',
 'MiscVal']

In [5]:
train_data = train_data.drop(columns=bad_columns)
train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,Utilities,Neighborhood,Condition1,Condition2,...,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,AllPub,CollgCr,Norm,Norm,...,0,0,0,NaN,NaN,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,AllPub,Veenker,Feedr,Norm,...,0,0,0,NaN,NaN,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,AllPub,CollgCr,Norm,Norm,...,0,0,0,NaN,NaN,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,AllPub,Crawfor,Norm,Norm,...,0,0,0,NaN,NaN,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,AllPub,NoRidge,Norm,Norm,...,0,0,0,NaN,NaN,12,2008,WD,Normal,250000


In [6]:
X, y = dm.split_data_set_to_x_y(train_data, 'SalePrice')
print(X.shape, y.shape)

(1460, 73) (1460,)


### 2.2. Обработка пропущенных значений и нормализация данных через ColumnTransformer и Pipeline

In [7]:
# Получение числовых колонок
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns

# Получение нечисловых колонок (всех остальных)
non_numeric_columns = X.select_dtypes(exclude=['float64', 'int64']).columns

numeric_columns, non_numeric_columns

(Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
        'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
        'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
        'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
        'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
        'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
        'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MoSold',
        'YrSold'],
       dtype='object'),
 Index(['MSZoning', 'Street', 'Alley', 'Utilities', 'Neighborhood',
        'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
        'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
        'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
        'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir',
        'Electrical', 'KitchenQual', 'Functional'

In [8]:
# Создаем preprocessor с разными трансформерами для разных типов данных
preprocessor = ColumnTransformer(
    transformers=[
        # Для числовых - импутация средним
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
        ]), numeric_columns),
        
        # Для категориальных - сначала импутация, затем one-hot кодирование
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),  # 'constant': заполняет константой, которую можно указать через параметр fill_value
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), non_numeric_columns)
    ], 
    sparse_threshold=0,  # Это заставит возвращать dense матрицу вместо sparse
    remainder='drop'  # Все остальные столбцы не будут включены в результат; passthrough - оставить нетронутыми
)

### 2.3. Разбиение данных на обучающую и тестовую выборки

In [9]:
RANDOM_STATE = 42

In [10]:
# Катбуст сам выбирает даныне для валидации, поэтому отсекаем данные только для теста
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

# Проверка размера полученных наборов
print(f'Train size: {len(X_train)}')
print(f'Test size: {len(X_test)}')

Train size: 1022
Test size: 438


### 2.4. Подбираем гиперпараметры с помощью optuna
Ниже схема "идеального" мира. как надо делать. но могут быть нюансы, например, если изначально данных и так мало...

In [11]:
# ┌─────────────────────────────────────────┐
# │       Исходный набор данных (100%)      │
# └─────────────────────────────────────────┘
#                      │
#                      │ разделяем
#                      ▼
# ┌─────────────────────────┐    ┌──────────────────┐
# │ Обучающий набор (80-90%)│    │ Тестовый (10-20%)│
# └─────────────────────────┘    └──────────────────┘
#            │                             │
#            ▼                             │
# ┌───────────────────────────────┐        │
# │   Подбор гиперпараметров      │        │
# │   (кросс-валидация внутри     │        │
# │          Optuna)              │        │
# └───────────────────────────────┘        │
#            │                             │
#            ▼                             │
# ┌───────────────────────────────┐        │
# │ Финальная модель с лучшими    │        │
# │ параметрами (обучение на      │        │
# │     полном обучающем наборе)  │        │
# └───────────────────────────────┘        │
#            │                             ▼
#            └────────────────────────► Оценка качества
#                                       (только 1 раз!)

In [12]:
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 800, 2000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.03, 0.2),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'silent': True,
        'random_seed': RANDOM_STATE,
    }
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', CatBoostRegressor(**params))
    ])

    # При подборе параметров через Optuna используй кросс-валидацию внутри самой Optuna, чтобы модель не переобучилась под одно разбиение, если делать просто оценку через RMSE без cross_val_score, то результаты будут завышены
    
    # Передаем только X_train часть, потом протестируем на X_test
    # score = cross_val_score(pipeline, X_train, y_train, scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)
    # Передав выше лишь часть данных, получил сильную просадку в качестве, поэтому передаем все данные, возможно, если бы данных было много изначально, то можно было бы передавать часть данных
    score = cross_val_score(pipeline, X, y, scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)
    return -score.mean()

In [13]:
# direction='minimize' - потому что мы хотим минимизировать RMSE. Т.е. в direction задается, что мы хотим сделать с целевой метрикой в процессе экспериментов - максимизировать ее или минимизировать. Минимизируем, например, ошибки. Максимизируем, например, точность.
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)  # n_trials - количество итераций (изменений гиперпараметров)

[I 2025-03-26 16:00:32,099] A new study created in memory with name: no-name-1c8240af-1a94-4739-9d2f-4d4b17bffae4
[I 2025-03-26 16:01:15,013] Trial 0 finished with value: 28060.503398641846 and parameters: {'iterations': 1862, 'depth': 10, 'learning_rate': 0.15164284826591956, 'l2_leaf_reg': 9.573472207895584, 'bagging_temperature': 0.9908393796255972}. Best is trial 0 with value: 28060.503398641846.
[I 2025-03-26 16:01:18,145] Trial 1 finished with value: 25848.48366528706 and parameters: {'iterations': 1191, 'depth': 5, 'learning_rate': 0.08974271467117986, 'l2_leaf_reg': 3.7715730984143856, 'bagging_temperature': 0.10658165626346527}. Best is trial 1 with value: 25848.48366528706.
[I 2025-03-26 16:01:26,367] Trial 2 finished with value: 27794.816516079787 and parameters: {'iterations': 1785, 'depth': 7, 'learning_rate': 0.19428149209524823, 'l2_leaf_reg': 1.4590191674350446, 'bagging_temperature': 0.02404819293092131}. Best is trial 1 with value: 25848.48366528706.
[I 2025-03-26 16:

In [14]:
best_params = study.best_params
print("Лучшие гиперпараметры:", best_params)
print("Лучшие значения метрики:", study.best_value)  


Лучшие гиперпараметры: {'iterations': 1098, 'depth': 4, 'learning_rate': 0.04000102325817965, 'l2_leaf_reg': 1.0885027090339108, 'bagging_temperature': 0.4668814661901983}
Лучшие значения метрики: 24922.21406237376


## 3. Обучение модели
Нашли лучшие параметры, обучаем модель

In [12]:
# параметры, полученные без cross_val_score, переобученные
# optuna_best_params = {
#     'iterations': 761, 
#     'depth': 5, 
#     'learning_rate': 0.0574601909986607, 
#     'l2_leaf_reg': 1.6382426806331774, 
#     'bagging_temperature': 0.6309506369801465
# }

# параметры, получены с cross_val_score
optuna_best_params = {'iterations': 1098, 'depth': 4, 'learning_rate': 0.04000102325817965, 'l2_leaf_reg': 1.0885027090339108, 'bagging_temperature': 0.4668814661901983}


In [13]:
cat_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', CatBoostRegressor(**optuna_best_params, silent=True))
])

# Обучение модели
cat_pipeline.fit(X_train, y_train)

# Предсказание и оценка
y_pred_cat = cat_pipeline.predict(X_test)
mse_cat = mean_squared_error(y_test, y_pred_cat)
rmse_cat = root_mean_squared_error(y_test, y_pred_cat)
r2_cat = r2_score(y_test, y_pred_cat)

mean_target = np.mean(y_test)

print("\n=== CatBoost ===")
print(f'RMSE: {rmse_cat:.4f}')
print(f'R² (коэффициент детерминации): {r2_cat:.4f}')
print(f'Относительная среднеквадратичная ошибка (Relative RMSE): {rmse_cat/mean_target*100:.2f}%')

# Используем кросс-валидацию (cross-validation), чтобы результаты были надежными и не зависели от случайного разбиения.
scores = cross_val_score(cat_pipeline, X, y, scoring='neg_root_mean_squared_error', cv=5)
rmse_scores = -scores
print("Средний RMSE на кросс-валидации:", rmse_scores.mean())


=== CatBoost ===
RMSE: 23544.0617
R² (коэффициент детерминации): 0.9206
Относительная среднеквадратичная ошибка (Relative RMSE): 13.08%
Средний RMSE на кросс-валидации: 25282.7399368738


Лучший результат до этого
- R² (коэффициент детерминации): 0.9224
- Относительная ошибка (RMSE/mean): 12.93%
- Средний RMSE на кросс-валидации: 25867.99820262165

In [14]:
cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scores = cross_val_score(cat_pipeline, X, y, scoring='neg_root_mean_squared_error', cv=cv)
rmse_scores = -scores
print("Средний RMSE на стабильной кросс-валидации:", rmse_scores.mean())

Средний RMSE на стабильной кросс-валидации: 27138.466608929688


## 4. Формируем Submission файл для Kaggle обучив модель на 100% данных

In [15]:
cat_pipeline.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(sparse_threshold=0,
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', '...
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir',
       'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC',
       'Fence', 'SaleType', 'SaleCondition'],
      dtype='object'))])),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x11a9bdf70>)])

In [16]:
X_submission = test_data.drop(columns=bad_columns)
X_submission.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,Utilities,Neighborhood,Condition1,Condition2,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,AllPub,NAmes,Feedr,Norm,...,0,0,120,0,NaN,MnPrv,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,AllPub,NAmes,Norm,Norm,...,0,0,0,0,NaN,NaN,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,AllPub,Gilbert,Norm,Norm,...,0,0,0,0,NaN,MnPrv,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,AllPub,Gilbert,Norm,Norm,...,0,0,0,0,NaN,NaN,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,AllPub,StoneBr,Norm,Norm,...,0,0,144,0,NaN,NaN,1,2010,WD,Normal


In [17]:
y_submission = cat_pipeline.predict(X_submission)
y_submission

array([120080.77010654, 165453.01219465, 189411.67536968, ...,
       167093.55369236, 123101.27262782, 223647.0150884 ])

In [18]:
submission = pd.DataFrame({
    'Id': test_data['Id'],
    'SalePrice': y_submission
})
submission.set_index('Id', inplace=True)
submission.head()

,SalePrice
Id,
1461,120080.770107
1462,165453.012195
1463,189411.675370
1464,193155.601784
1465,186056.390384


In [19]:
submission.to_csv('submission_optuna_tuned_v3.csv')

# Выводы
Гипероптимизация с optuna штука интересная. 
На вход ей лучше подавать (в идеальном мире) - в функцию оценки, не весь дата-сет, а лишь его часть, оставив что-то для финальной валидации модели после подбора параметров.
В самой функции для оценки лучше использовать не чистый RMSE, а cross_val, чтобы избежать оверфиттинга на одном разбиении данных.
Но какие бы эксперименты не ставил, никакой из результирующих наборов получавшихся гиперпараметров не смог побить результат самого первого, дефолтного, в лоб взятого набора из итерации improved_realization:
(iterations=1000, 
learning_rate=0.05, 
depth=6, 
loss_function='RMSE', 
verbose=0, 
random_seed=RANDOM_STATE,
), что довольно забавно, учитывая, что в той итерации на подбор гиперпараметров было затрачено примерно 0 минут, а сейчас затрачено несоклько часов экспериментов и улучшений - никаких.